# Baselines

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from baselines import get_baseline
from baselines import multi_class_roc_auc, multi_class_spearman
from baselines import empirical_dist
from pprint import pprint
import pandas as pd
import numpy as np

/Users/ellerywulczyn/miniconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
d = pd.read_csv('../../data/annotations/baseline/annotations.tsv', sep='\t')
d.index = d.rev_id

In [3]:
d_b = d.query("sample=='blocked'")
d_r = d.query("sample=='random'")

In [4]:
iters = 10
K = 20
tasks = ['attack', 'recipient', 'aggression']
metrics = {'ROC': multi_class_roc_auc, 'spearman':multi_class_spearman}
ds = {'blocked': d_b, 'random':d_r}

In [5]:
def get_all_baselines(K, tasks, metrics, ds, iters, pairs, n_jobs = 8):
    df_results = []
    results = {}
    
    for task in tasks:
        results[task] = {}
        for d_name, d in ds.items():
            results[task][d_name] = {}
            for metric_name, metric in metrics.items():
                results[task][d_name][metric_name] = {pair:[] for pair in pairs}
                l = d[task]
                for i in range(iters):
                    pms = get_baseline(l, K, empirical_dist, metric, pairs, n_jobs=n_jobs)
                    for pair, m in pms.items():
                        results[task][d_name][metric_name][pair].append(m)
                        
                        
                        
    for task in tasks:
        print('Task: ', task)
        for d_name, d in ds.items():
            print('\tData: ', d_name)
            for metric_name, metric in metrics.items():
                print('\t\tMetric: ', metric_name)
                for pair in pairs:
                    a = np.array(results[task][d_name][metric_name][pair])
                    print('\t\t\t(%d, %d): %0.3f (%0.3f)' % (pair[0], pair[1], a.mean(), a.std()))
                    df_results.append({'task':task,
                                       'data_type': d_name,
                                       'metric': metric_name,
                                       'n_pred': pair[0],
                                       'n_gold': pair[1],
                                       'score': a.mean(),
                                       'error': a.std(),
                                          })
    pd.DataFrame(df_results).to_csv('baselines.csv', index=False)

In [6]:
F = int(K/2)
pairs = list(zip(range(1, F+1), range(1, F+1))) + list(zip(range(1, F+1), [F]*F))

In [7]:
get_all_baselines(K, tasks, metrics, ds, iters, pairs, n_jobs=4)

Task:  attack
	Data:  blocked
		Metric:  spearman
			(1, 1): 0.466 (0.009)
			(2, 2): 0.612 (0.009)
			(3, 3): 0.673 (0.007)
			(4, 4): 0.712 (0.005)
			(5, 5): 0.744 (0.005)
			(6, 6): 0.764 (0.004)
			(7, 7): 0.781 (0.003)
			(8, 8): 0.795 (0.003)
			(9, 9): 0.812 (0.004)
			(10, 10): 0.818 (0.002)
			(1, 10): 0.597 (0.009)
			(2, 10): 0.690 (0.006)
			(3, 10): 0.732 (0.004)
			(4, 10): 0.760 (0.004)
			(5, 10): 0.778 (0.002)
			(6, 10): 0.790 (0.002)
			(7, 10): 0.801 (0.002)
			(8, 10): 0.809 (0.003)
			(9, 10): 0.815 (0.003)
			(10, 10): 0.818 (0.002)
		Metric:  ROC
			(1, 1): 0.738 (0.007)
			(2, 2): 0.863 (0.006)
			(3, 3): 0.884 (0.004)
			(4, 4): 0.928 (0.005)
			(5, 5): 0.933 (0.002)
			(6, 6): 0.954 (0.002)
			(7, 7): 0.954 (0.002)
			(8, 8): 0.965 (0.002)
			(9, 9): 0.966 (0.001)
			(10, 10): 0.973 (0.002)
			(1, 10): 0.877 (0.005)
			(2, 10): 0.932 (0.002)
			(3, 10): 0.934 (0.003)
			(4, 10): 0.956 (0.002)
			(5, 10): 0.954 (0.002)
			(6, 10): 0.965 (0.002)
			(7, 10): 0.

In [8]:
#get_all_baselines(4, ['aggression',], {'ROC': multi_class_roc_auc}, {'blocked':ds['blocked']}, 2, [(1,3)])